In [1]:
import spacy
import textblob
import os
import re
import string
import csv
from textblob import Word
from textblob import TextBlob
from collections import Counter

In [2]:
# initialize spacy
nlp = spacy.load('en')

path = '../corpus/airport/chapters/'

# grammatical categories
# these are clearNLP labels
# 'agent' catches by, need to add logic to catch passive subjects
np_labels = set(['nsubj','nsubjpass', 'csubj', 'csubjpass'])

# filter out what, that, this etc.
blacklist = ['what','that', 'this', 'who', 'which', 'one', 'all', 'these', 'you']



In [3]:
# let's add another pass with corefernce resolution

subjects = []
# ingest text in parts, the new spacy parser chokes
# on large files otherwise
for filename in os.listdir(path):
    if filename.endswith(".resolved"):
        with open(path + filename, 'r') as myreadfile:
            spacy_doc = nlp(myreadfile.read())

            # filter as much as possible 
            # but retain the diff mr X and mrs x
            for chunk in spacy_doc.noun_chunks:
                for token in chunk:
                    if token.lemma_ != '-PRON-' and token.dep_ in np_labels:
                        if token.lemma_ not in blacklist:
                            s = chunk.string.lower()
                            #filter(s.isalpha, string.printable)
                            #stripped = ''.join(filter(str.isalnum, s))
                            newstring = ''

                            for char in s:
                                if char.isalpha() or char == ' ':
                                    newstring = newstring + char

                            subjects.append(newstring.strip())

In [4]:
# process the list

#for subject in subjects:
#    blob = TextBlob(subject)
#    for word in blob.words:
#        print(word)
#        synsets = Word(word).synsets
#        for synset in synsets:
#            print(word, synset.lexname())

subj_counter = Counter(subjects)

subj_counter.most_common()


[('mel', 245),
 ('keith', 156),
 ('vernon', 145),
 ('gwen', 94),
 ('patroni', 87),
 ('tanya', 84),
 ('guerrero', 80),
 ('cindy', 59),
 ('mrs quonsett', 44),
 ('d o guerrero', 36),
 ('the plane', 35),
 ('elliott freemantle', 34),
 ('harris', 33),
 ('keith bakersfeld', 25),
 ('peter coakley', 23),
 ('a man', 22),
 ('everyone', 21),
 ('elliott', 21),
 ('the baby', 21),
 ('inez', 20),
 ('ordway', 19),
 ('the passengers', 17),
 ('your husband', 17),
 ('the airport', 16),
 ('that mel', 16),
 ('people', 16),
 ('another man', 16),
 ('nobody', 15),
 ('the man', 15),
 ('tevis', 14),
 ('perry yount', 13),
 ('the agent', 13),
 ('tomlinson', 12),
 ('zanetta', 12),
 ('the old lady', 12),
 ('danny', 12),
 ('weatherby', 12),
 ('roberta', 12),
 ('the people', 11),
 ('ada quonsett', 11),
 ('bunnie', 11),
 ('jordan', 11),
 ('mel bakersfeld', 10),
 ('the storm', 10),
 ('someone', 10),
 ('the policeman', 10),
 ('vernon demerest', 10),
 ('the girl', 10),
 ('natalie', 10),
 ('the men', 10),
 ('captain harris

In [5]:
with open('list-agents.csv', 'w') as write_csv:
    writer = csv.writer(write_csv, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    for row in subj_counter.most_common():
        writer.writerow(row)